In [2]:
!pip install awswrangler

  Using cached awswrangler-2.4.0-py3-none-any.whl (171 kB)
  Using cached redshift_connector-2.0.874-py3-none-any.whl (81 kB)
  Using cached pg8000-1.17.0-py3-none-any.whl (34 kB)
  Using cached PyMySQL-1.0.2-py3-none-any.whl (43 kB)
  Using cached pandas-1.2.2-cp37-cp37m-manylinux1_x86_64.whl (9.9 MB)
  Using cached pytz-2021.1-py2.py3-none-any.whl (510 kB)
  Using cached scramp-1.2.2-py3-none-any.whl (7.8 kB)
  Using cached requests-2.25.1-py2.py3-none-any.whl (61 kB)
  Using cached asn1crypto-1.4.0-py2.py3-none-any.whl (104 kB)
ERROR: pg8000 1.17.0 has requirement scramp==1.2.0, but you'll have scramp 1.2.2 which is incompatible.
  Attempting uninstall: pytz
    Found existing installation: pytz 2019.3
    Uninstalling pytz-2019.3:
      Successfully uninstalled pytz-2019.3
  Attempting uninstall: asn1crypto
    Found existing installation: asn1crypto 1.3.0
    Uninstalling asn1crypto-1.3.0:
      Successfully uninstalled asn1crypto-1.3.0
  Attempting uninstall: requests
    Found e

In [3]:
import awswrangler as wr

In [4]:
import getpass
bucket = getpass.getpass()
path = f"s3://{bucket}/data/"

KeyboardInterrupt: 

In [3]:
if "awswrangler_test" not in wr.catalog.databases().values:
    wr.catalog.create_database("awswrangler_test")

In [4]:
df = wr.athena.read_sql_query("SELECT * FROM event_etl limit 100", database="default", categories=["event_name","event_datetime","product_id","mbr_no"])

In [5]:
df.head()

,event_name,event_datetime,product_id,mbr_no
0,abx:list_view,"""2020-07-27 07:46:19.228""",CF31135013,12344157
1,abx:list_view,"""2020-07-27 07:46:19.228""",CF31135013,12344157
2,abx:list_view,"""2020-07-27 07:46:19.228""",CF31135013,12344157
3,abx:list_view,"""2020-07-27 07:46:19.228""",CF31135013,12344157
4,abx:list_view,"""2020-07-27 07:46:19.228""",CF31135013,12344157


In [6]:
df.event_datetime = df.event_datetime

In [9]:
import pandas as pd

In [10]:
#df = df.applymap(lambda x: x.replace('"', ''))
df.event_datetime = pd.to_datetime(df.event_datetime, infer_datetime_format=True).astype('int')

In [11]:
df.columns = ['EVENT_TYPE','TIMESTAMP','ITEM_ID','USER_ID']

In [12]:
!pip install s3fs
import s3fs

     |████████████████████████████████| 7.2 MB 2.4 MB/s eta 0:00:01
ERROR: boto3 1.17.12 has requirement botocore<1.21.0,>=1.20.12, but you'll have botocore 1.19.52 which is incompatible.
ERROR: awscli 1.19.12 has requirement botocore==1.20.12, but you'll have botocore 1.19.52 which is incompatible.
  Attempting uninstall: botocore
    Found existing installation: botocore 1.20.12
    Uninstalling botocore-1.20.12:
      Successfully uninstalled botocore-1.20.12


In [13]:
df.head()

,EVENT_TYPE,TIMESTAMP,ITEM_ID,USER_ID
0,abx:list_view,1597862344848000000,CF31135896,11437564
1,abx:list_view,1597862344848000000,CF31135896,11437564
2,abx:list_view,1597862344848000000,CF31135896,11437564
3,abx:list_view,1597862344848000000,CF31135896,11437564
4,abx:list_view,1597862344848000000,CF31135896,11437564


In [20]:
bytes_to_write = df.to_csv(None,header=True,index=False).encode()
s3 = s3fs.S3FileSystem(anon=False)
with s3.open('s3://everything-kanto/ldcs/interactions/interactions.csv','wb') as f:
    f.write(bytes_to_write)